# Ingestion template
Ingest raster data to SentinelHub

TODO
- [ ] convert raster to COG: add script to convert raster into COG, either within this notebook or as separate script (or else?)
- [ ] are there alternatives to ingesting tile one by one? (cell 2)
- [ ] add ingestion routine for multitemporal collections


In [ ]:
# Configure plots for inline use in Jupyter Notebook
%matplotlib inline

import datetime as dt

# Utilities
import boto3
import dateutil
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os
import rasterio

# Sentinel Hub
from sentinelhub import (
    CRS,
    BBox,
    ByocCollection,
    ByocCollectionAdditionalData,
    ByocCollectionBand,
    ByocTile,
    DataCollection,
    DownloadFailedException,
    MimeType,
    SentinelHubBYOC,
    SentinelHubRequest,
    SHConfig,
    bbox_to_dimensions,
    os_utils,
)

config = SHConfig()
config.instance_id = os.environ.get("SH_INSTANCE_ID")
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")
config.aws_access_key_id = os.environ.get("username")
config.aws_secret_access_key = os.environ.get("password")    

In [ ]:
# Initialize SentinelHubBYOC class
byoc = SentinelHubBYOC(config=config)

# !! change collection name
new_collection = ByocCollection(name="collection_name", s3_bucket="hub-fairicube0")
#byoc.delete_collection(new_collection) # for the deleting of the collection
created_collection = byoc.create_collection(new_collection)
my_collection = byoc.get_collection(created_collection["id"])


#https://sentinelhub-py.readthedocs.io/en/latest/examples/byoc_request.html#Create-new-collection

input_folder ="data/path/to/data"

# add tiles one by one
tile_list = [
        "tile_1_(BAND).tif",
        "tile_2_(BAND).tif",
        "tile_3_(BAND).tif",
        "..."
            ]

# ingest tiles
for tile in tile_list:
    path_link = input_folder+"/"+tile
    new_tile = ByocTile(path=path_link, , sensing_time=dt.datetime(2018, 1, 1)) # change to correct date
    created_tile = byoc.create_tile(created_collection, new_tile)
    tile = byoc.get_tile(collection=created_collection["id"], tile=created_tile["id"])
    tile

In [ ]:
# check ingestion status
print("Collection name:", my_collection["name"])
print("Collection id: ", my_collection["id"])
tiles = list(byoc.iter_tiles(collection))
for tile in tiles:
    print("Tile status: ", tile['status'])
    print("Tile created: ", tile['created'])
    if(tile['status'] == "FAILED"):
        print("Ingestion failed error: ", tile['additionalData'])
print("-------------------")